## Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error , r2_score
import warnings
warnings.filterwarnings('ignore')
import joblib

## Load The Dataset

In [2]:
df= pd.read_csv("First Model GP Project.csv")

In [3]:
df.head()

,Unnamed: 0,age,weight(kg),height(m),gender,BMI,BMR,activity_level,calories_to_maintain_weight,BMI_tags,Label
0,1,2,16.097862,0.932025,F,18.531612,958.584,1.2,1150.3008,8,13
1,2,4,14.619374,0.916687,F,17.397496,932.383,1.7,1585.0511,7,1
2,4,4,17.899918,0.997288,F,17.997414,977.578,1.9,1857.3982,7,4
3,6,3,13.532880,1.022786,F,12.936609,944.689,1.9,1794.9091,7,3
4,8,4,17.039484,1.053977,M,15.338909,799.229,1.9,1518.5351,7,0


## Removing Unnecessary Columns

In [4]:
df= df.drop(["Unnamed: 0","BMI_tags","Label"],axis=1)

## Convert Categorical Gender Column to Numercal

In [5]:
df = pd.get_dummies(df, columns=['gender'], drop_first=False)

### Traning The model By 
#### Dividing data into Features "X" and target value "Y" 
#### Splitting data into Training and Testing
#### Using The model LinearRegression
#### Finally Evaluating The model by R2 Score and Mean Squared error

In [6]:
def train_calorie_model(df):
    X = df.drop(['calories_to_maintain_weight'], axis=1)
    y = df['calories_to_maintain_weight']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"Model R2 Score: {r2_score(y_test, y_pred):.4f}")
    print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.2f}")
    return model

### Saving The model

In [7]:
model = train_calorie_model(df)
joblib.dump(model, "calorie_model.pkl")

Model R2 Score: 0.9874
Mean Squared Error: 1966.03


['calorie_model.pkl']

## Calculate BMI 

In [8]:
def calculate_bmi(weight_kg, height_m):
    return weight_kg / (height_m ** 2)

## Calculate BMR

In [9]:
def calculate_bmr(age, weight_kg, height_m, gender_F, gender_M):
    if gender_F == 1:  # Female
        return 655 + (9.6 * weight_kg) + (1.8 * height_m * 100) - (4.7 * age)
    elif gender_M == 1:  # Male
        return 66 + (13.7 * weight_kg) + (5 * height_m * 100) - (6.8 * age)
    else:
        raise ValueError("Invalid gender values")

## Calculate Calories

In [10]:
def predict_calories(user_inputs, model):
    bmi = calculate_bmi(user_inputs['weight'], user_inputs['height'])
    bmr = calculate_bmr(user_inputs['age'], user_inputs['weight'], user_inputs['height'],
                        user_inputs['gender_F'], user_inputs['gender_M'])
    
    features = np.array([[user_inputs['age'], user_inputs['weight'], user_inputs['height'],
                          bmi, bmr, user_inputs['activity_level'],
                          user_inputs['gender_F'], user_inputs['gender_M']]])
    
    predicted_calories = model.predict(features)[0]

    if bmi < 18.5:
        predicted_calories += 300  
    elif bmi >= 30:
        predicted_calories -= 300  
    return round(predicted_calories)

### Example 

In [11]:
user_input = {
        "age": 2,
        "weight": 16,
        "height": 0.94,
        "activity_level": 1.2,
        "gender_F": 1,
        "gender_M": 0
    }

User_Calories = predict_calories(user_input, model)
print(f"Predicted Daily Calories: {User_Calories} kcal")

Predicted Daily Calories: 1227 kcal
